In [1]:
from pipeline import Pipeline
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="Pikurrot/yolo11n-licenseplates", filename="yolo11n_licenseplates.pt")
pipeline = Pipeline(model_path)

/home/eric/miniconda3/envs/yolo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device cuda


In [2]:
segments = pipeline("images/reduced/")


image 1/12 /media/eric/D/repos/ANPR-GIA/images/reduced/0.jpg: 384x640 1 license_plate, 45.7ms
image 2/12 /media/eric/D/repos/ANPR-GIA/images/reduced/1.jpg: 384x640 1 license_plate, 14.5ms
image 3/12 /media/eric/D/repos/ANPR-GIA/images/reduced/10.jpeg: 480x640 1 license_plate, 36.2ms
image 4/12 /media/eric/D/repos/ANPR-GIA/images/reduced/11.jpeg: 640x480 1 license_plate, 35.4ms
image 5/12 /media/eric/D/repos/ANPR-GIA/images/reduced/2.jpg: 288x640 1 license_plate, 34.8ms
image 6/12 /media/eric/D/repos/ANPR-GIA/images/reduced/3.jpg: 288x640 1 license_plate, 12.7ms
image 7/12 /media/eric/D/repos/ANPR-GIA/images/reduced/4.jpg: 288x640 1 license_plate, 12.8ms
image 8/12 /media/eric/D/repos/ANPR-GIA/images/reduced/5.jpg: 384x640 1 license_plate, 14.6ms
image 9/12 /media/eric/D/repos/ANPR-GIA/images/reduced/6.jpg: 480x640 1 license_plate, 9.6ms
image 10/12 /media/eric/D/repos/ANPR-GIA/images/reduced/7.jpg: 640x640 1 license_plate, 12.2ms
image 11/12 /media/eric/D/repos/ANPR-GIA/images/reduced

In [4]:
detections = pipeline.detect("images/reduced/", 640, 0.25, -1)


image 1/12 /media/eric/D/repos/ANPR-GIA/images/reduced/0.jpg: 384x640 1 license_plate, 15.4ms
image 2/12 /media/eric/D/repos/ANPR-GIA/images/reduced/1.jpg: 384x640 1 license_plate, 14.3ms
image 3/12 /media/eric/D/repos/ANPR-GIA/images/reduced/10.jpeg: 480x640 1 license_plate, 10.4ms
image 4/12 /media/eric/D/repos/ANPR-GIA/images/reduced/11.jpeg: 640x480 1 license_plate, 9.2ms
image 5/12 /media/eric/D/repos/ANPR-GIA/images/reduced/2.jpg: 288x640 1 license_plate, 13.7ms
image 6/12 /media/eric/D/repos/ANPR-GIA/images/reduced/3.jpg: 288x640 1 license_plate, 12.5ms
image 7/12 /media/eric/D/repos/ANPR-GIA/images/reduced/4.jpg: 288x640 1 license_plate, 13.1ms
image 8/12 /media/eric/D/repos/ANPR-GIA/images/reduced/5.jpg: 384x640 1 license_plate, 15.0ms
image 9/12 /media/eric/D/repos/ANPR-GIA/images/reduced/6.jpg: 480x640 1 license_plate, 9.3ms
image 10/12 /media/eric/D/repos/ANPR-GIA/images/reduced/7.jpg: 640x640 1 license_plate, 12.0ms
image 11/12 /media/eric/D/repos/ANPR-GIA/images/reduced/

In [5]:
detections

[[{'image': <PIL.Image.Image image mode=RGB size=3x4032>,
   'box': [1237, 1200, 2557, 1568],
   'conf': 0.8641365170478821}],
 [{'image': <PIL.Image.Image image mode=RGB size=3x4032>,
   'box': [1056, 937, 2003, 1314],
   'conf': 0.8627058863639832}],
 [{'image': <PIL.Image.Image image mode=RGB size=3x4032>,
   'box': [1885, 1289, 2675, 1782],
   'conf': 0.9310530424118042}],
 [{'image': <PIL.Image.Image image mode=RGB size=3x3024>,
   'box': [1720, 1888, 1913, 2332],
   'conf': 0.8982633352279663}],
 [{'image': <PIL.Image.Image image mode=RGB size=3x2560>,
   'box': [697, 626, 1373, 783],
   'conf': 0.8348332047462463}],
 [{'image': <PIL.Image.Image image mode=RGB size=3x2560>,
   'box': [996, 473, 1449, 628],
   'conf': 0.5373973846435547}],
 [{'image': <PIL.Image.Image image mode=RGB size=3x2560>,
   'box': [926, 654, 1467, 810],
   'conf': 0.682855486869812}],
 [{'image': <PIL.Image.Image image mode=RGB size=3x4032>,
   'box': [533, 742, 1465, 1216],
   'conf': 0.887658417224884}]